In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [51]:
df = pd.read_csv("winequality-red.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [52]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [53]:
df["alcohol_cat"] = pd.cut(df["alcohol"],
                            bins=[ 8, 9, 10, 11, 12, np.inf],
                            labels=[1,2,3,4,5])

## 특성추가

In [54]:
df['sulph_sub_chlor']=df['sulphates']-df['chlorides'] 
df['alchol_sub_vol']=df['alcohol']-df['volatile acidity']  
df['alchol_usb_ph']=df['alcohol']-df['pH']  
df['alcohol_sub_crit_acid']=df['alcohol']-df['citric acid']  
df['alcohol_sub_alcho_sub_vol']=df['alcohol']-df['alchol_sub_vol'] 
df['alcohol_sum_vol']=df['alcohol']+df['volatile acidity']  
df['alcohol_sum_ph']=df['alcohol']+df['pH'] 
df['alcohol_sum_crit_acid']=df['alcohol']+df['citric acid'] 
df['alcohol_sum_alcho_sub_vol']=df['alcohol']+df['alchol_sub_vol']  
df['alcohol_mul_vol']=df['alcohol']*df['volatile acidity']  
df['alcohol_mul_ph']=df['alcohol']*df['pH'] 
df['alcohol_mul_crit_acid']=df['alcohol']*df['citric acid']  
df['alcohol_mul_alcho_sub_vol']=df['alcohol']*df['alchol_sub_vol']  

In [55]:
conrr_matrix = df.corr()
conrr_matrix["quality"].sort_values(ascending=False)

quality                      1.000000
alchol_sub_vol               0.517468
alcohol_sum_crit_acid        0.499493
alcohol_sum_alcho_sub_vol    0.498843
alcohol_mul_alcho_sub_vol    0.495742
alchol_usb_ph                0.494159
alcohol                      0.476166
alcohol_cat                  0.462620
alcohol_sum_ph               0.450025
alcohol_sub_crit_acid        0.436264
alcohol_sum_vol              0.418953
alcohol_mul_ph               0.376588
sulph_sub_chlor              0.307735
alcohol_mul_crit_acid        0.292002
sulphates                    0.251397
citric acid                  0.226373
fixed acidity                0.124052
residual sugar               0.013732
free sulfur dioxide         -0.050656
pH                          -0.057731
chlorides                   -0.128907
density                     -0.174919
total sulfur dioxide        -0.185100
alcohol_mul_vol             -0.261368
alcohol_sub_alcho_sub_vol   -0.390558
volatile acidity            -0.390558
Name: qualit

In [56]:
df.index

RangeIndex(start=0, stop=1599, step=1)

## 데이터 셔플

In [57]:
from sklearn.utils import shuffle

df = shuffle(df)

In [58]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'alcohol_cat',
       'sulph_sub_chlor', 'alchol_sub_vol', 'alchol_usb_ph',
       'alcohol_sub_crit_acid', 'alcohol_sub_alcho_sub_vol', 'alcohol_sum_vol',
       'alcohol_sum_ph', 'alcohol_sum_crit_acid', 'alcohol_sum_alcho_sub_vol',
       'alcohol_mul_vol', 'alcohol_mul_ph', 'alcohol_mul_crit_acid',
       'alcohol_mul_alcho_sub_vol'],
      dtype='object')

## 이상치 대체 하기

In [59]:
import numpy as np

def replace_outliers_with_mean(df):
    '''replaces outliers with the mean value for specific columns'''
    variables = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'sulph_sub_chlor',
       'alchol_sub_vol', 'alchol_usb_ph', 'alcohol_sub_crit_acid',
       'alcohol_sub_alcho_sub_vol', 'alcohol_sum_vol', 'alcohol_sum_ph',
       'alcohol_sum_crit_acid', 'alcohol_sum_alcho_sub_vol', 'alcohol_mul_vol',
       'alcohol_mul_ph', 'alcohol_mul_crit_acid', 'alcohol_mul_alcho_sub_vol']
    
    for variable in variables:
        mean_value = df[variable].mean()
        std_value = df[variable].std()
        outliers = (df[variable] - mean_value).abs() > 3 * std_value
        df.loc[outliers, variable] = mean_value
        
    return df

In [60]:
df_outliers = replace_outliers_with_mean(df)

C:\Users\xmflr\AppData\Local\Temp\ipykernel_15748\177021386.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.6360225140712945' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[outliers, variable] = mean_value


In [61]:
X_train = df_outliers.drop(["quality", "fixed acidity", "density"], axis=1)
y_train = df_outliers["quality"]


In [62]:
def display_score(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [67]:
from sklearn.model_selection import cross_val_score

In [64]:
from sklearn.ensemble import ExtraTreesRegressor

ext_reg = ExtraTreesRegressor(n_estimators=500, max_depth=30, random_state=42, n_jobs=-1, max_features=0.7)
ext_reg_score = cross_val_score(ext_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
ext_rmse_score = np.sqrt(-ext_reg_score)
display_score(ext_rmse_score)

Scores: [0.58960853 0.52179142 0.55635308 0.56418593 0.47184119 0.59108097
 0.51958076 0.51586288 0.51640126 0.49690886]
Mean: 0.5343614889887662
Standard deviation: 0.03741015098908544


# 10-CV Result
## Mean: 0.5343614889887662

In [66]:
print(X_train.shape)

(1599, 23)
